In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

from Project.Database import Db

In [38]:
year1_hour, meta1_hour  = Db.load_data(year=1, hourly=True, meta=True)
year2_hour, meta2_hour = Db.load_data(year=2, hourly=True, meta=True)

In [39]:
production = []
consumption = []

for df in [year1_hour, year2_hour]:
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%Y-%m-%d %H:%M:%S%z", utc=True) - pd.to_timedelta(unit="h", arg=(df["Timestamp"].str.split(pat="-", expand=True)[lambda self: self.columns[-1]].str[1]).astype(int))
    df["Season"] = "summer"
    df.loc[(pd.DatetimeIndex(df["Timestamp"]).month.isin([3,4,5])), "Season"] = "spring"
    df.loc[(pd.DatetimeIndex(df["Timestamp"]).month.isin([9,10,11])), "Season"] = "fall"
    df.loc[(pd.DatetimeIndex(df["Timestamp"]).month.isin([1,2,12])), "Season"] = "winter"
    production.append(meta1_hour.loc[(meta1_hour['Parameter'] == "Power_Electrical") & (
    meta1_hour["Description"].str.contains("produced"))]["Unnamed: 0"].tolist())

In [29]:
consumption_condition = (lambda self: ((self['Parameter'] == "Power_Electrical") | (self['Parameter'] == "Power_Thermal")) & (
    self["Description"].str.contains("power consumption" or "used")))

production_condition = (lambda self: (self['Parameter'] == "Power_Electrical") & (
    self["Description"].str.contains("produced")))

production_year1 = meta1_hour.loc[production_condition, "Unnamed: 0"].tolist()

consumption_year1 = meta1_hour.loc[consumption_condition, "Unnamed: 0"].tolist()

production_year2 = meta2_hour.loc[production_condition, "Unnamed: 0"].tolist()

consumption_year2 = meta2_hour.loc[consumption_condition, "Unnamed: 0"].tolist()

In [6]:
year1_hour["Production"] = year1_hour[production_year1].sum(1)
year1_hour["Consumption"] = year1_hour[consumption_year1].sum(1)

In [7]:
year2_hour["Production"] = year2_hour[production_year2].sum(1)
year2_hour["Consumption"] = year2_hour[consumption_year2].sum(1)

In [8]:
year1_hour["Season_order"] = [0 if Season == "spring" else 1 if Season == "summer" else 2 if Season == "fall" else 3 for Season in year1_hour["Season"]]
year1_hour.sort_values(by="Season_order", ascending=True, inplace=True)

In [9]:
year2_hour["Season_order"] = [0 if Season == "spring" else 1 if Season == "summer" else 2 if Season == "fall" else 3 for Season in year2_hour["Season"]]
year2_hour.sort_values(by="Season_order", ascending=True, inplace=True)

In [10]:
# Seasonal violin plots of production for both year 1 and 2
fig = px.violin(year1_hour, x="Season", y="Production", color="Season")
fig.write_html(Db.save_file_directory("Season_production_violin_year1.html"))

fig = px.violin(year2_hour, x="Season", y="Production", color="Season")
fig.write_html(Db.save_file_directory("Season_production_violin_year2.html"))

In [11]:
# Seasonal violin plots of consumption for both year 1 and 2
fig = px.violin(year1_hour, x="Season", y="Consumption", color="Season")
fig.write_html(Db.save_file_directory("Season_consumption_violin_year1.html"))

fig = px.violin(year2_hour, x="Season", y="Consumption", color="Season")
fig.write_html(Db.save_file_directory("Season_consumption_violin_year2.html"))

In [28]:
consumption_loads_condition = (lambda self: ((self['Parameter'] == "Power_Electrical") | (self['Parameter'] == "Power_Thermal")) & (
    self["Description"].str.contains("power consumption" or "used")) & (self["Subsystem"] == "Loads"))

consumption_loads_year1 = meta1_hour.loc[consumption_loads_condition, "Unnamed: 0"].tolist()

consumption_loads_year2 = meta2_hour.loc[consumption_loads_condition, "Unnamed: 0"].tolist()
consumption_loads_year1

['Elec_PowerChildADownstairs',
 'Elec_PowerChildAUpstairs',
 'Elec_PowerChildBDownstairs',
 'Elec_PowerChildBUpstairs',
 'Elec_PowerParentADownstairs',
 'Elec_PowerParentAUpstairs',
 'Elec_PowerParentBDownstairs',
 'Elec_PowerParentBUpstairs',
 'Load_1stFloorLightsPowerUsage',
 'Load_1stFloorSensHeatPowerUsage',
 'Load_2ndFloorLightsPowerUsage',
 'Load_2ndFloorSensHeatPowerUsage',
 'Load_BR2PlugLoadsPowerUsage',
 'Load_BR3PlugLoadsPowerUsage',
 'Load_BR4PlugLoadsPowerUsage',
 'Load_ClothesWasherPowerWithStandby',
 'Load_DryerPowerTotal',
 'Load_KPlugLoadsPowerUsage',
 'Load_LRPlugLoadsPowerUsage',
 'Load_LatentHeatPowerUsage',
 'Load_MBRPlugLoadsPowerUsage',
 'Load_MicrowavePowerWithStandby',
 'Load_OvenPowerTotal',
 'Load_RefrigeratorPowerWithStandby']

In [40]:
fig = px.bar(year1_hour, x="Season", y=sum(consumption_loads_year1), color=consumption_loads_year1)
fig.show()

TypeError: unsupported operand type(s) for +: 'int' and 'str'